In [ ]:
# Gyakran változtatott konstansok

# Stop word-ök használata
USE_STOPWORDS = True

# Elérési úton lévő mentett modell betöltése kiértékelésre
LOAD_CHECKPOINT = False

# Az optimális validációs veszteséggel rendelkező epoch-ban lévő modellel való visszatérés tanításból
RESTORE_BEST_WEIGHTS = True

# GPU használata. Érték változtatása esetén a Kernel újraindítandó
USE_GPU = True

# Checkpoint könyvtár alatti könyvtár
CHECKPOINT_SUBDIR = ''

# Könyvtár, mely a CHECKPOINT_SUBDIR alatt található. Ide történik a kiértékelés mentése, illetve innen töltődik be
# a kiértékelendő modell. Ha a könyvtár nem létezik, akkor létrehozásra kerül.
CHECKPOINT_PREFIX = 'c_sw'


In [ ]:
# Importok

import os
import pickle
import re

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sn
import tensorflow as tf
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.callbacks import EarlyStopping
from transformers import TFBertForSequenceClassification, AutoTokenizer

In [ ]:
# Konstansok

# Random seed-ek értéke
SHUFFLE_RANDOM_STATE = 42
TRAIN_RANDOM_STATE = 42
TEST_RANDOM_STATE = 42

# A program számára fontos fejlécek értéke az adatbázisban
TEXT = 'Sentence'
Y_HEADER = 'LABEL'

# Az adatbázisban található értékek megfeleltetése a programban
LABELS = {
    "SPORT": 0,
    "VIDEÓJÁTÉK": 1
}

# Maximális tokensorozat méret
MAX_SEQUENCE_LENGTH = 64

# Tanítás során alkalmazott batch méret
BATCH_SIZE = 16

# Tanítás során alkalmazott epoch szám. Korai megállás esetén nem feltétlenül jut el a megadott értékig a program
EPOCHS = 10

# Maximum hány TRAIN dokumentum kerüljön tokenizálásra. None esetén mind
TRAIN_PROCESSED_MAX_DOCUMENTS = None

# Maximum hány TEST dokumentum kerüljön tokenizálásra. None esetén mind
TEST_PROCESSED_MAX_DOCUMENTS = None

In [ ]:
# GPU beállítása

if not USE_GPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
if physical_devices:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
# Mappastruktúra beállítása

path = 'checkpoints/' + CHECKPOINT_SUBDIR + CHECKPOINT_PREFIX

ITER = "1"
if not os.path.exists(path):
  os.makedirs(path)

In [ ]:
# Szöveg feldolgozására alkalmas metódusok

URL_RE = 'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}([-a-zA-Z0-9()@:%_+.~#?&/=]*)'
WHITELIST_RE = '[^a-zA-Z0-9íÍöÖüÜóÓőŐúÚáÁéÉűŰ]'

def cleanse(i):
    """
    Adatbázisban szereplő dokumentumok megtisztítása az URL-ektől és a nem-alfanumerikus karakterektől.

    Parameters
    ----------
    i: int
        A tisztítandó dokumentumnak az indexe.

    Returns
    -------
    str
        A megtisztított dokumentum.
    """

    text = dataset[TEXT].iloc[i]
    text = re.sub(URL_RE, ' ', text)
    text = re.sub(WHITELIST_RE, ' ', text)
    text = ' '.join([text])
    return re.sub(' +', ' ', text)

def delete_empty_rows(dataset):
    """
    Az adatbázist megtisztítja az üres soroktól.

    Parameters
    ----------
    dataset: pandas.DataFrame
        Az adatbázisból készített DataFrame.

    Returns
    -------
    pandas.DataFrame
        A megtisztított adatbázis.
    """

    ids_to_delete = dataset.index[dataset[TEXT] == ' '].tolist()
    return dataset.drop(ids_to_delete)

In [ ]:
# Adatok előkészítése

dataset = pd.read_csv('db/sport_or_e_sport.csv', sep=';', error_bad_lines=False)
dataset[Y_HEADER] = dataset[Y_HEADER].map(LABELS)
dataset = shuffle(dataset, random_state=SHUFFLE_RANDOM_STATE)
dataset.info()

dataset = delete_empty_rows(dataset)
for i  in range(len(dataset.index)):
    dataset[TEXT].iloc[i] = cleanse(i)

dataset.head(5)
X = dataset[TEXT].values
y = dataset[Y_HEADER].values

X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.8, random_state=TRAIN_RANDOM_STATE)

X_dev, X_test, y_dev, y_test = train_test_split(X_rem, y_rem, train_size=0.5, random_state=TEST_RANDOM_STATE)

y_train_labels = y_train
y_dev_labels = y_dev
y_test_labels = y_test
y_train = to_categorical(y_train, 2)
y_dev = to_categorical(y_dev, 2)
y_test = to_categorical(y_test, 2)


In [ ]:
# Adatok vizualizálása

def plot_label_counts(y, title='y labels'):
    """
    Adatok eloszlásának ábrázolása.

    Parameters
    ----------
    y: numpy.ndarray
        Az adathalmaz címkéi, amelyre a vizualizálás történik.
    title: str
        Az elkészített ábra címe.
    """

    unique, counts = np.unique(y, return_counts=True)
    b = dict(zip(unique, counts))
    plt.barh(range(len(b)),  list(b.values()), align='center', color=['lightblue', 'lightgreen'])
    y_values = ["Sport", "Videójáték"]
    y_axis = np.arange(0, 2, 1)
    plt.yticks(y_axis, y_values)
    plt.title(title)
    plt.xlabel('Number of Samples in training Set')
    plt.ylabel('Label')
    ax = plt.gca()
    for i, v in enumerate(b.values()):
        plt.text(ax.get_xlim()[1]/100, i, str(v), color='blue', fontweight='bold')
    plt.show()


plot_label_counts(y_train_labels, 'Train eloszlas')
plot_label_counts(y_dev_labels, 'Dev eloszlas')
plot_label_counts(y_test_labels, 'Test eloszlas')


In [ ]:
# huBERT betöltése a Hugging Faces Model Hub-ból

bert_tokenizer = AutoTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")
bert_model = TFBertForSequenceClassification.from_pretrained("SZTAKI-HLT/hubert-base-cc", num_labels=2)

In [ ]:
# Stop word-ök letöltése

nltk.download('stopwords')

In [ ]:
# Stop word-ök alkalmazásának előkészítése

all_stopwords = []

def apply_stopwords(sentences):
    """
    Stop word-ök alkalmazása magyar nyelvű korpuszra.

    Parameters
    ----------
    sentences: list of numpy.ndarray
        Az adathalmazban található dokumentumokat tartalmazó lista, amelyen a stop word-ök alkalmazva lesznek.

    Returns
    -------
    list of str
        A stop word-öket már nem tartalmazó dokumentumokat tartalmazó lista.
    """

    global all_stopwords
    corpus = []
    for sen in sentences:
        sentence = sen.split()
        all_stopwords = stopwords.words('hungarian')
        whitelist = ["ne", "nem", "se", "sem"]
        sentence = [word for word in sentence if (word.lower() not in all_stopwords or word.lower() in whitelist)
                 and len(word) > 1]
        sentence = ' '.join(sentence)
        corpus.append(sentence)

    return corpus


In [ ]:
# Dokumentumok tokenizálása

def batch_encode(X):
    """
    Tokenizálás a huBERT tokenizálóval.

    Parameters
    ----------
    X: list of str
        Az adathalmazban található dokumentumok listája, amelyek tokenizálásra kerülnek.

    Returns
    -------
    transformers.tokenization_utils_base.BatchEncoding
        A tokenizált dokumentumok.
    """

    return bert_tokenizer.batch_encode_plus(
    X,
    truncation=True,
    max_length=MAX_SEQUENCE_LENGTH,
    add_special_tokens=True, # add [CLS] and [SEP] tokens
    return_attention_mask=True,
    return_token_type_ids=False, # not needed for this type of ML task
    padding='max_length', # add 0 pad tokens to the sequences less than max_length
    return_tensors='tf'
)
X_train = X_train.tolist() if not USE_STOPWORDS else apply_stopwords(X_train.tolist())
X_dev = X_dev.tolist() if not USE_STOPWORDS else apply_stopwords(X_dev.tolist())
X_test = X_test.tolist() if not USE_STOPWORDS else apply_stopwords(X_test.tolist())


X_train = batch_encode(X_train[:TRAIN_PROCESSED_MAX_DOCUMENTS])
X_dev = batch_encode(X_dev[:TEST_PROCESSED_MAX_DOCUMENTS])
X_test = batch_encode(X_test[:TEST_PROCESSED_MAX_DOCUMENTS])
y_train = y_train[:TRAIN_PROCESSED_MAX_DOCUMENTS]
y_dev = y_dev[:TEST_PROCESSED_MAX_DOCUMENTS]
y_test = y_test[:TEST_PROCESSED_MAX_DOCUMENTS]

In [ ]:
# Modell létrehozása

def create_model():
    """
    Modell létrehozása, melynek inputja az input id-kat, illetve az attention mask-ot tartalmazó
    réteg, tartalmazza a BERT modellt, kimeneti rétege osztályozára alkalmas.

    Returns
    -------
    keras.engine.functional.Functional
        Az elkészített modell.
    """

    input_ids = tf.keras.layers.Input(shape=(64,), dtype=tf.int32, name='input_ids')
    attention_mask = tf.keras.layers.Input((64,), dtype=tf.int32, name='attention_mask')
    output = bert_model([input_ids, attention_mask])[0]
    output = tf.keras.layers.Dropout(rate=0.15)(output)
    output = tf.keras.layers.Dense(2, activation='softmax')(output)
    result = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)
    return result

model = create_model()

opt = tf.keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Modell adatainak kiiratása

print(bert_model.config)
model.summary()

In [ ]:
# Checkpoint callback beállítása a modell checkpontjainak lementéséhez

checkpoint_path = path + 'cp.ckpt'
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
# Tanítás során alkalmazott metódusok

def get_history_as_text(history, epoch):
    """
    Visszaadja a kiírandó és fájlba mentendő stringet, amely a tanítás history-ját tartalmazza.

    Returns
    -------
    str
        A kiírandó és fájlba mentendő stringet, amely a tanítás history-ját tartalmazza.
    """

    return f'Epoch {epoch+1: <3}: loss: {format(history["loss"][epoch], ".4f")} - accuracy: {format(history["accuracy"][epoch], ".4f")} - val_loss: {format(history["val_loss"][epoch], ".4f")} - val_accuracy: {format(history["val_accuracy"][epoch], ".4f")}'

def fit_model():
    """
    Modell illesztését elvégző metódus. Amennyiben a LOAD_CHECKPOINT True, úgy illesztés helyett a mentett tanítást
    tölti be és annak eredményével tér vissza.

    Returns
    -------
    history: dict of fit results
        Az illesztés során az epoch-ok információit tartalmazó dic, mely tartalmazza a
        loss, az accuracy, a val_loss és a val_accuracy adatokat.
    result: list of float
        A modell kiértékelését tartalmazza a teszt adathalmazra.
    predict: numpy.ndarray
        A modell predikcióit tartalmazza a teszt adathalmazra.
    """

    if not LOAD_CHECKPOINT:
        early_stopping_callback = EarlyStopping(
            monitor='val_loss',
            mode='min',
            verbose=1,
            patience=2,
            restore_best_weights=RESTORE_BEST_WEIGHTS)
        if ITER == "6":
            history = model.fit(
                x=X_train.values(),
                y=y_train,
                validation_data=(X_dev.values(), y_dev),
                epochs=EPOCHS,
                batch_size=BATCH_SIZE,
                callbacks=[early_stopping_callback, cp_callback]
            )
        else:
            history = model.fit(
                x=X_train.values(),
                y=y_train,
                validation_data=(X_dev.values(), y_dev),
                epochs=EPOCHS,
                batch_size=BATCH_SIZE,
                callbacks=[early_stopping_callback]
            )
        with open(path + '.history', 'wb') as file_pi:
            pickle.dump(history.history, file_pi)
        text_file = open(path + "history" + ITER + ".txt", "w")
        print(len(history.history['loss']))
        for i in range(len(history.history['loss'])):
            history_text = get_history_as_text(history.history, i)
            print(history_text)
            text_file.writelines(history_text + '\n')
        text_file.close()
    else:
        model.load_weights(checkpoint_path)
        history = pickle.load(open(path + '.history', "rb"))
        for i in range(len(history['loss'])):
            history_text = get_history_as_text(history, i)
            print(history_text)
    result = model.evaluate(X_test.values(), y_test)
    predict = model.predict(X_test.values())
    np_predict = np.argmax(predict,axis=1)
    return history, result, np_predict

In [ ]:
# Kiértékelés vizualizálása szövegesen és ábrán

le = LabelEncoder()

def evaluate(predict, result, y):
    """
    A modell kiértékelése. Kiíratja az osztályozási eredményeket, a pontosságot, illetve az igazságmátrixot.

    Parameters
    ----------
    predict: numpy.ndarray
        A modell predikcióit tartalmazza a teszt adathalmazra.
    result: list of float
        A modell kiértékelését tartalmazza a teszt adathalmazra.
    y: numpy.ndarray
        A teszt adathalmaz címkéit tartalmazza.
    """

    y_le = le.fit_transform(y[:TEST_PROCESSED_MAX_DOCUMENTS])
    print('Classification report:')
    print(classification_report(y_le, predict))
    print(f'Accuracy: {accuracy_score(y_le, predict): >43}')
    print(f'Accuracy from evaluation: {result[1]: >27}')
    print('Confusion matrix:')
    df_cm = pd.DataFrame(confusion_matrix(y_le, predict),
                         index=[i for i in ['sport', 'videójáték']],
                         columns=[i for i in ['sport', 'videójáték']])
    if not LOAD_CHECKPOINT:
        with open(path + "results" + ITER + ".txt", "w") as text_file:
            df_cm_string = df_cm.to_string(header=False, index=False)
            text_file.write('Classification report:\n')
            text_file.write(classification_report(y_le, predict))
            text_file.write(f'\nAccuracy: {accuracy_score(y_le, predict): >43}\n')
            text_file.write(f'Accuracy from evaluation: {result[1]: >27}\n')
            text_file.write('\nConfusion matrix:\n')
            text_file.write(df_cm_string)
    plt.figure(figsize=(10,7))
    plt.title(CHECKPOINT_PREFIX[:-1])
    hm = sn.heatmap(df_cm, annot=True, fmt='g', cmap="Blues")
    hm.set(ylabel='True label', xlabel='Predicted label')
    if not LOAD_CHECKPOINT:
        plt.savefig(path + 'accuracy-' + format(result[1], ".4f") + ITER + '.jpg')
    plt.show()


In [ ]:
# Modell illesztése és kiértékelése 6 iterációban.

for i in range(1, 7):
    ITER = str(i)
    print("Iteration ", ITER)
    history, result, predict = fit_model()
    evaluate(predict, result, y_test_labels)